In [ ]:
!pip install g2p-en regex

# Goals

My goals with this notebook is to provide some grounding for spelling in silabex.
Each silabex character is constructed from:

1) an initial consonant cluster which can have any 1 of 1024 combinations.
2) a vowel cluster which can have any of 16 combinations.
3) an optional final consonant which can also have any 1 of 1024 combinations.

In stenography, the system this writing system was original based on, 
the 3 parts roughly correspond to: the `left hand`, the `vowel keys` at the thumb, and the `right hand`.  
There are some differences between the two however, 
so I feel like `silabex` should have it's own spelling theory sepearte from `steno`.

### spelling priorities

My overall priorities when designing this spelling system are as follows:

* Prioritize American english pronounciation. As this is my first langauge, I feel I can do a better job with this as the basis
* Develop this spelling system to cleave more closely to modern pronunciation.  
   It's my understanding that english pronunciation changed quite a bit 
   even after spellings started to standarize. [why-is-english-spelling-so-tricky](https://blog.duolingo.com/why-is-english-spelling-so-tricky/)  
   It feels like this is a good a time as any to update standard spellings
   since this writing system is so different from standard english.
* Compress the most common words into as few characters as possible.
   Even the english alphabet does not accurately represent the sounds required to reproduce a word. [chameleon](https://www.youtube.com/watch?v=-Fy_NYCtSgw)  
   For the most common words, sacrificing accuracy for brevity seems like a reasonable trade off.
   This priority does conflict with other priorities however, so it should likely only apply to a few thousand of the most common words at most.
* Redundent graphmemes should be destroyed. kbye

### what this project does not do
/
In order to prevent this project from becoming much larger,
there are a few things that I don't intend to do included here:

Out of Scope:

* Replace aribic numerals with another numbering system.
   It's an interesting idea to play around with, but it's beyond the scope of this project.
* Fully standardize spelling in some meaningful way.
   The goal is mostly to get a good start and provide some solid footing.
   Fleshing out the spelling system is a much larger and much longer project to take on.
* Seriously consider non-english words and how to handle them.
   English is currently all over the place on this (Hors d'oeuvre anyone?) and other writing systems like Japanese have entier different writing systems for foreign words.
   Integrating foreign, and loan words, as well as non english nouns into the writing system will be a whole different process.
   

# Silabex

Silabex is a new writing system for english that adopts an eastern athsetic and attempts to create more standard representations of words
All characters are 1:1 and consist of the following structure:

```sh
 -----------v-----------
 |                     |
 |    /  /     /  /    |
 |                     |
 |    ----    ----     |
 |    |  |    |  |     |
 v    -I--    --F-     v
 |    |  |    |  |     |
 |    ----    ----     |
 |                     |
 |    /  /    /  /     |
 |                     |
 -----------v-----------
 ```

where the outer lines (marked by v) represent vowel sounds, the left set of strokes represent the initial consonant cluster and the right set of strokes represent the final consonant cluster.

## Resources

There are several things I want to take into consideration while doing this research:
1) Phonemes - these are the `atoms of pronuciation`. These are particularly important for determining how to split up characters in `silabex`
2) Graphmemes - these are the `atoms of spelling` (it's not just letters, combinations like sh, or ch are also technically atoms in this sense)

#### NLTK

nltk looks like it has a lot of useful tools that could help me analyse textual data

#### cmudict

Provides a method for converting words into phonemes.
Can be found here https://www.nltk.org/_modules/nltk/corpus/reader/cmudict.html

#### G2P-EN

This looks like a great way to convert words into graphmemes.
g2p-en can be found here https://pypi.org/project/g2p-en/

#### Stenography

This was the original inspiration for the `silabex` writing system.
It splits words down into sylablx adjacent pieces and has a fairly limited keyboard with translates decently to character designs
It's also what lead to the idea of breaking a character down into and initial consonant cluster, a vowel cluster, and an optional final consonant cluster.

#### plover

[Plover](https://www.openstenoproject.org/plover/) is the most open project providing information on stenography (which is a suprizingly closed off eco-system).

My initial idea was to steal the plover steno theory directly, but I have a feeling that with a little bit of work I can come up with a system more suited to my writing system.
The main point is that while in stento the S, and * keys are doubled up, that's not a necessary feature to carry over to `silabex`. 

# Phonemes

Let's start by investigating phoneme frequencies.
It seems like g2p uses the phoneme descriptors defined in the [Carnegie Mellon Pronouncing Dictionary](https://www.nltk.org/_modules/nltk/corpus/reader/cmudict.html) so I want to see how frequent each of those phonemes tends to be.
It also looks like this repo [dolph/dictionary](https://github.com/dolph/dictionary) is an easy place to start since I just have to basically use the text file.
Let's clone that repo down and do some analisys.

Looks like we can also grab all the words that were used in the Carnegie Mellon Dictionary so let's try doing that as well.

### Analysis

So it looks like there are a total of 15 vowels and 24 consonant phonemes.
That's great becuase I can represent a total of 16 vowels and way more than 24 consonants.
I know that consonants can appear in clusters though so I need to figure out what cluster are the most common.
It also looks like most phonemes have an exponential fall of in how frequently they are used which was expected.

I'm also realizing that the average American probalby only ever activly uses about 40k words max:
* https://blog.cyracom.com/ciiblog/the-lifelong-pursuit-of-language-learning-how-the-vocabularies-of-native-and-non-native-speakers-compare
* https://www.economist.com/johnson/2013/05/29/lexical-facts

My dataset might honestly be too large.
I should focus on improving the quality of word choice rather than trying to find more words to analyse.

In [2]:
# force a fresh clone of the dolph/dictionary
!rm -rf data/dictionary
!git clone https://github.com/dolph/dictionary data/dictionary

Cloning into 'data/dictionary'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Receiving objects: 100% (45/45), 1.89 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
import numpy as np

common_words = np.empty(0)
for file in ["dictionary/popular.txt"]:
    with open(f"data/{file}", "r") as f:
        print(f"importing {file}...")
        raw_words = f.read()
        new_words = [ word.lower() for word in raw_words.split("\n") ]
        print(f"got {len(new_words)} words")

    common_words = np.append(common_words, new_words)

common_words = np.unique(common_words)
print("Final Unique Word Total: ", common_words.shape[0])
common_words[:10]

importing dictionary/popular.txt...
got 25323 words
Final Unique Word Total:  25323


array(['', 'aa', 'aardvark', 'aargh', 'aback', 'abacus', 'abandon',
       'abandoned', 'abandoning', 'abandonment'], dtype='<U32')

In [ ]:
from g2p_en import G2p
import numpy as np
from tqdm import tqdm
from nltk.corpus import cmudict
import re

g2p = G2p()
phoneme_list = np.empty(0)
simple_phoneme_list = np.empty(0)
for word in tqdm(common_words, desc="Building Phoneme Lists"):
    if word == "":
        continue

    out = g2p(word)
    phoneme_list = np.append(phoneme_list, out)
    simple_phoneme_list = np.append(simple_phoneme_list, [ re.sub('\d+', "", i) for i in out ] )

cmudict_words = cmudict.dict()
for word in tqdm(cmudict_words, desc="Add cmudict Words"):
    if word == "":
        continue

    out = cmudict_words[word][0]
    phoneme_list = np.append(phoneme_list, out)
    simple_phoneme_list = np.append(simple_phoneme_list, [ re.sub('\d+', "", i) for i in out ] )
    
display(np.unique(phoneme_list))
display(np.unique(simple_phoneme_list))

# save these so we don't have to recompute them every time
with open("data/phoneme_list.npy", "wb") as f:
    np.save(f, phoneme_list)

with open("data/simple_phoneme_list.npy", "wb") as f:
    np.save(f, simple_phoneme_list)

Add cmudict Words:  13%|█████████▌                                                              | 16391/123455 [09:30<1:33:02, 19.18it/s]

# Analyse the Data

now that we've loaded the data all into memory we need to analyse it


In [ ]:
with open("data/phoneme_list.npy", "rb") as f:
    phoneme_list = np.load(f)

with open("data/simple_phoneme_list.npy", "rb") as f:
    simple_phoneme_list = np.load(f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(30, 6))
plt.hist(phoneme_list, bins='auto')
plt.title(f"Phoneme Frequency In {common_words.shape[0]} Common Words")
plt.xlabel("phoneme")
plt.ylabel("count")
plt.show()

In [ ]:
def show_phoneme_chart(phonemes, title, x_axis, y_axis):
    p, counts = np.unique(phonemes, return_counts=True)
    zipped = list(zip(p.tolist(), counts.tolist()))

    sorted_phonemes = np.flip(
        np.sort(
            np.array(zipped, dtype=[("phoneme", "U8"),("count", int)]),
            axis=0,
            order="count",
        )
    )

    plt.figure(figsize=(30, 6))
    plt.bar(*zip(*sorted_phonemes))
    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.show()


In [ ]:
show_phoneme_chart(
    phonemes=phoneme_list,
    title=f"Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis="phoneme", 
    y_axis="count",
)

In [ ]:
show_phoneme_chart(
    phonemes=simple_phoneme_list,
    title=f"Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted, No Vowel Emphasis)", 
    x_axis="phoneme", 
    y_axis="count",
)

In [ ]:
vowel_list = [ phoneme for phoneme in phoneme_list if re.match('.*\d', phoneme) ]

show_phoneme_chart(
    phonemes=vowel_list,
    title=f"Vowel Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis="vowel phoneme", 
    y_axis="count",
)

In [ ]:
simple_vowel_list = [ re.sub('\d', "", vowel) for vowel in vowel_list ]

show_phoneme_chart(
    phonemes=simple_vowel_list,
    title=f"Vowel Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)",
    x_axis=f"vowel phoneme (total: {len(np.unique(simple_vowel_list))})", 
    y_axis="count",
)

In [ ]:
unique_simple_vowel_list = np.unique(simple_vowel_list)
consonant_list = [ phoneme for phoneme in simple_phoneme_list if not np.isin(phoneme, unique_simple_vowel_list) ]

show_phoneme_chart(
    phonemes=consonant_list, 
    title=f"Consonant Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis=f"consonant phoneme (total: {len(np.unique(consonant_list))})", 
    y_axis="count",
)

# Consonant Clusters

In english consonants often appear next to each other, I should probably take some time trying to sus out the most common consonant clusters as well as the most common individual clusters

### Analysis
* Vowels can be the `primary stressed sound` (1) the `secondary stressed sound` (2) or an `unstressed sound` (0) in a word
* Vowels that are stressed, seems to "take" the vowels surrounding them more than unstressed vowels
* Consonants that appear at the beginning or end of a word seem to always clump together
* "S" seems to stick to vowels more than consonants when in side words (it's almost like that "s" sound likes to slide into a nice vowel phoneme)

Here are a few unique words I've found:

* Puritan ('P', 'Y', 'UH1', 'R', 'AH0', 'T', 'AH0', 'N') the first 'AH0' gives up the 'R' to 'UH1' but also give up the 'T' to the second 'AH0'.
  does that mean that internal unemphasized vowels are the least likely to clump with their consonant neighbors.
* Forsaking ('F', 'AO0', 'R', 'S', 'EY1', 'K', 'IH0', 'NG') why don't the 'R' and 'S' clump here? It seems like the 'RS' composit cluster is potentially uncommon internally in a word.
  I know it's not that weird at the end for words (force for example ends with 'RS')
* Mildly ('M', 'AY1', 'L', 'D', 'L', 'IY0') Chosing how to split up 'L', 'D', 'L' is interesting.
  My guess is that 'LD' is a pretty common consonant clusters (old, gold, mold), and maybe more common than 'DL' (fiddle, muddle)?


In [ ]:
from g2p_en import G2p
import numpy as np
import re

def is_vowel(phoneme):
    return re.match('.*\d', phoneme) is not None

def is_consonant(phoneme):
    return not is_vowel(phoneme)

def initial_consonant_cluster(phoneme_list):
    cluster = []
    for phoneme in phoneme_list:
        if not is_consonant(phoneme):
            break
        cluster.append(phoneme)

    return tuple(cluster)

def final_consonant_cluster(phoneme_list):
    vowel_point = -1
    final = []
    for i, phoneme in enumerate(phoneme_list):
        if not is_consonant(phoneme):
            vowel_point = i
            final = []
            continue
            
        final.append(phoneme)

    if vowel_point == -1:
        return ()

    return tuple(final)

g2p = G2p()
consonant_clusters_list = []
for word in tqdm(common_words, desc="Building Phoneme Lists"):
    if word == "":
        continue
        
    phoneme_list = g2p(word)
    
    initial = initial_consonant_cluster(phoneme_list)
    if len(initial) > 0:
        consonant_clusters_list.append(initial)

    final = final_consonant_cluster(phoneme_list)
    if len(final) > 0:
        consonant_clusters_list.append(final)

consonant_clusters = np.array(consonant_clusters_list, dtype=object)
print(len(consonant_clusters))
np.unique(consonant_clusters)

In [ ]:
dipthongs = [ "/".join(phoneme) for phoneme in consonant_clusters if len(phoneme) == 2 ]

show_phoneme_chart(
    phonemes=dipthongs, 
    title=f"Diphthongs Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis=f"diphthong phoneme (total: {len(np.unique(dipthongs))})", 
    y_axis="count",
)

In [ ]:
tripthongs = [ "/".join(phoneme) for phoneme in consonant_clusters if len(phoneme) == 3 ]

show_phoneme_chart(
    phonemes=tripthongs, 
    title=f"Triphthongs Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis=f"triphthongs phoneme (total: {len(np.unique(tripthongs))})", 
    y_axis="count",
)

In [ ]:
quadriphthong = [ "/".join(phoneme) for phoneme in consonant_clusters if len(phoneme) == 4 ]

show_phoneme_chart(
    phonemes=quadriphthong, 
    title=f"Quadriphthong Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
    x_axis=f"quadriphthong phoneme (total: {len(np.unique(quadriphthong))})", 
    y_axis="count",
)

In [ ]:
cluster5 = [ "/".join(phoneme) for phoneme in consonant_clusters if len(phoneme) > 4 ]
print("there are no clusters longer than 3 phonemes: cluster5.length =", len(cluster5))

# show_phoneme_chart(
#     phonemes=cluster5, 
#     title=f"Long Consonant Cluster Phoneme Frequency In {common_words.shape[0]} Common Words (Sorted)", 
#     x_axis=f"consonant cluster (total: {len(np.unique(cluster5))})", 
#     y_axis="count",
# )

# Consonant Cluster Order

It's clear to me that there's a pretty dramatic preference for certin phoneme clusters over others.
Given that there are less than 100 diphthongs and triphthongs out of the possible 576 and 13,824 respectivly, not all clusters are created equal.
The spelling system then should have a strong preference for representing common english sounds rather than trying to support all possible consonant clusters.


In [ ]:
def show_nth_consonant_chart(index, ordinal):
    data = [ phoneme[index] for phoneme in consonant_clusters if len(phoneme) > index ]
    show_phoneme_chart(
        phonemes=data,
        title=f"{ordinal} Phoneme Frequency In Consonant Clusters In {common_words.shape[0]} Common Words (Sorted)",
        x_axis=f"(consonant (total: {len(np.unique(data))})", 
        y_axis="count",
    )

show_nth_consonant_chart(0, "First")
show_nth_consonant_chart(1, "Second")
show_nth_consonant_chart(2, "Third")
show_nth_consonant_chart(3, "Fourth")

# Consonant Cluster Corelation

How often do given consonant phonemes appear with on another in consonant clusters in 

# Analysis

So it looks like the figure i've seen that english has about 250 total unique consonant clusters is probably right.
It's also crazy how much more often S and T appear together than any other consonant combination.

In [ ]:
import re

def draw_matrix(matrix, title, zero_identity=False):
    
    draw_matrix = np.zeros(shape=matrix.shape, dtype=float)
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            draw_matrix[x,y] = matrix[x,y]
        if zero_identity:
            draw_matrix[y, y] = 0
            
    plt.figure(figsize=(12, 12))
    plt.imshow(draw_matrix, interpolation='none')
    plt.title(title)
    plt.colorbar(orientation='vertical')
    plt.xticks(np.arange(0, count), labels=unique_consonant_list)
    plt.yticks(np.arange(0, count), labels=unique_consonant_list)
    plt.show()

# get all the consonant phonemes sorted by frequency
consonants, counts = np.unique([ consonant for consonant in consonant_clusters if len(consonant) == 1 ], return_counts=True)
zipped_consonants_list = list(zip(consonants.tolist(), counts.tolist()))
sorted_consonants_list = np.flip(
    np.sort(
        np.array(zipped_consonants_list, dtype=[("consonant", "U8"),("count", int)]),
        axis=0,
        order="count",
    )
)

unique_consonant_list = np.array([ consonant[0] for consonant in sorted_consonants_list ])
count = unique_consonant_list.shape[0]
np_matrix = np.zeros(shape=(count, count), dtype=float)

for cluster in consonant_clusters:
    for i, c0 in enumerate(cluster):
        for c1 in cluster[i:]:
            x = np.where(unique_consonant_list == (c1))[0]
            y = np.where(unique_consonant_list == (c0))[0]
            np_matrix[x,y] += 1
            np_matrix[y,x] += 1

# do some quick validation
for y in range(count):
    for x in range(count):
        assert np_matrix[x,y] <= np_matrix[y,y], f"m[{x}, {y}] = {np_matrix[x,y]} but m[{y}, {y}] = {np_matrix[y,y]}"
        assert np_matrix[x,y] == np_matrix[y,x], f"m[{x}, {y}] = {np_matrix[x,y]} but m[{y}, {x}] = {np_matrix[y,x]}"
            
draw_matrix(np_matrix, "Consonant Correlation within Clusters", zero_identity=True)

# normalize the matrix base on the diaganal
np_matrix_norm = np.zeros(shape=(count, count), dtype=float)
for y in range(count):
    for x in range(count):
        np_matrix_norm[x,y] = np_matrix[x,y] / np_matrix[y,y]

    # quick validation check again
    assert np_matrix_norm[y,y] == 1.0

draw_matrix(np_matrix_norm, "Normalized Consonant Correlation within Clusters", zero_identity=True)

# identify zeros
total = 0.0
non_zero = 0.0
np_matrix_bin = np.zeros(shape=(count, count), dtype=float)
for y in range(count):
    for x in range(count):
        total += 1
        if np_matrix[x,y] > 0.01:
            non_zero += 1
            np_matrix_bin[x,y] = 1

draw_matrix(np_matrix_bin, f"Binary Consonant Correlation within Clusters ({(non_zero/total)*100:.2f}% covered)")

# Encoding Strategies

One of the key things to take into account when crafting an encoding is to measure how effective the encoding is comunicating english words.
This is espeically important given how complex english consonant clusters can get.
For examples, in the 25k common words I found 1 consonant cluster that included 6 unique consonant sounds.
Therefor, a good written language will need to support the complex nature of english consonant clusters.

There are a couple metrics I want to use to judge how good a given encoding is for this representation.
I can represent my written system using a series of point so developing an abstract representation of the encoding first and then inform the visual design.

### distinctivness
This measures how distinct all the given consonant cluster encodings are from each other.
In the best case, all consonant clustesrs are represented by totally distinct encodings.
In the worst case, all consonant clusters are represented by the exact same encoding.
Distinctiveness is calculated using the following formulas:

\begin{align*}
D(E, C) &= \frac{\sum_{w \in C} U(w, E, C)}{\sum \#P(w)} \\
U(w, E, C) &= \sum_{p \in P(w)} Q(p) \\
Q(p) &= \begin{cases}
    0 & \text{if } E(p) \in \bigcup \{ E(p_0) \mid p_0 \in N(p, C) \} \\
    1 & \text{otherwise}
\end{cases} \\
N(p, C) &= \bigcup \{ P(w) | w \in C \} - p
\end{align*}

Where D calculates the distinctivness of an encoder E given a corpus of text C.
E is the encoding function which gives and encoding value for each phoneme.
P(w) returns *JUST* the consonat phonemes of a given word, consonant clusters are represented by a single phonemes.
U(w, E, C) is the uniquness score of a word(w) given an encoder(E) and a text corpus(C).
Q(p, C) is the uniquenes score of a single phonemes(p) encoding given a text corpus(C).
N(p, C) is the set of all unique phonemes in a text corpus(C) minus the given phoneme(p).
#a is the numer of element in the list a

### visual complexity
Visual complexity is a measure of how complex it is to parse a given consolant cluster.
Lower visual complexity is better especially for common clusters that appear often.
Visual complexity tends to increase as the number of points in an encoding increase, however sometimes denser representations can actually be more visually clear.
Visual complexity is calculated using the following formulas:

\begin{align*}
V(E, C) = Σ F(w, E) | w ∈ C  

F(w, E) = Σ G(E(p)) | p ∈ w 
\end{align*}

Where V(E, C) calculates the visual complexity of characters encoding given an encoder E, and a corpus of text C.
F(w, E) is the visual complexity of the word.
G(e) computs the visual complexity score for a given phoneme encoding.
E(p) takes a phoneme from a word and produces the encoding for that phoneme.

### visual audio coherence
Encodings may have good distintivness and visual clarity.
However, writen language reflect the physical reality of producing sound using our vocal cords.
Representing similar sounds with similar symboles will likely make the writing system easier to learn and use.
Visual audio coherence can be calculated using the following.

\begin{align*}
r = (\#T D_{ep} - D_e*D_p / \sqrt{ (\#T D_{es} - D_e^2) * (\#T D_{ps} - D_p^2)})
\end{align*}

\begin{align*}
D_e = Σ e(t0, t1) | t ∈ T
\end{align*}

\begin{align*}
D_es = Σ e(t0, t1)^2 | t ∈ T
\end{align*}

\begin{align*}
D_p = Σ p(t0, t1) | t ∈ T
\end{align*}

\begin{align*}
D_ps = Σ p(t0, t1)^2 | t ∈ T
\end{align*}

\begin{align*}
D_ep = Σ e(t0, t1) * p(t0, t1) | t ∈ T
\end{align*}

\begin{align*}
T = { (p0, p1) | p0 ∈ { p | p ∈ C }, p1 ∈ { p | p ∈ C } }
\end{align*}


`r` is the Pearson correlation coefficient and calculates the correlation between phoneme encoding and phoneme production.  
`e` calculates the distance between phoneme embeddings and can produce a vector for a more robust representation.  
`p` calculates the distance between phoneme production and can produce a vector for a more robust representation.  
`D_e` is the sum of all embedding distances for all tuples in T.  
`D_es` is the sume of all squared embedding distances for all tuples in T.  
`D_p` is the sum of all phoneme production distances for all tuples in T.  
`D_ps` is the sum of all phoneme production distances for all tuples in T.  
`T` is the set of all unique tuples where each tuple represents on possible combination of 2 phonemes from a given corpus of text.  


# Thoughts

A lot of these ideas can proabbly be tied back into information theory.
I should dig into that and see what I can find.

In [ ]:

# 0 - 1
# 2 - 3
# 4 - 5
# 6 - 7
# 8 - 9

mapping_1 = {
    "R": [0],
    "P": [1],
    "N": [2],
    "S": [3],
    "K": [4],
    "D": [5],
    "L": [6],
    "T": [7],
    "Z": [8],
    "M": [9],
    "B": [0, 2],
    "F": [1, 2],
    "H": [3, 2],
    "G": [4, 2],
    "V": [5, 2],
    "W": [6, 2],
    "J": [7, 2],
    "C": [8, 2],
    "Y": [9, 2],
}
    